In [143]:
import os
import yaml
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import json

Preprocessing

In [144]:
# Load and preprocess data
def load_data(directory):
    data = []
    labels = []
    
    for filename in os.listdir(directory):
        if filename.endswith(".yml"):
            with open(os.path.join(directory, filename), 'r') as yaml_file:
                hand_data = yaml.safe_load(yaml_file)
                landmarks = [point['x'] for point in hand_data['hand_landmarks']]
                data.append(landmarks)
                labels.append(directory.split("/")[-1])  # Assuming the class label is the last part of the directory

    return np.array(data), labels

In [145]:
# Prepare dataset
signs_dir = "Signs"
classes = os.listdir(signs_dir)
all_data = []
all_labels = []


In [146]:
# Convert list to dictionary
my_dict = {i: classes[i] for i in range(len(classes))}

# Save the dictionary to a JSON file
with open('labels_info.json', 'w') as json_file:
    json.dump(my_dict, json_file)

In [147]:
print(all_labels)
print(classes)

[]
['Hello', 'I love You', 'U', 'V', 'W']


In [148]:
for sign_class in classes:
    data, labels = load_data(os.path.join(signs_dir, sign_class))
    all_data.extend(data)
    all_labels.extend(labels)


In [149]:
# Convert labels to integers
label_to_int = {label: i for i, label in enumerate(set(all_labels))}
all_labels = [label_to_int[label] for label in all_labels]

In [150]:
print(label_to_int)

{'Signs\\W': 0, 'Signs\\I love You': 1, 'Signs\\U': 2, 'Signs\\V': 3, 'Signs\\Hello': 4}


Training

In [151]:
# Convert data to numpy array
X_data = np.array(all_data)
y_labels = np.array(all_labels)


In [152]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.2, random_state=42)


In [153]:
# Build the model
model = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(set(all_labels)), activation='softmax')
])

In [154]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [155]:
# Train the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100


25/25 [==============================] - 1s 11ms/step - loss: 1.6139 - accuracy: 0.2262 - val_loss: 1.6008 - val_accuracy: 0.2900
Epoch 2/100
25/25 [==============================] - 0s 5ms/step - loss: 1.5962 - accuracy: 0.3125 - val_loss: 1.5830 - val_accuracy: 0.3500
Epoch 3/100
25/25 [==============================] - 0s 5ms/step - loss: 1.5783 - accuracy: 0.2875 - val_loss: 1.5705 - val_accuracy: 0.5200
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 1.5588 - accuracy: 0.4450 - val_loss: 1.5441 - val_accuracy: 0.5250
Epoch 5/100
25/25 [==============================] - 0s 4ms/step - loss: 1.5256 - accuracy: 0.4087 - val_loss: 1.5158 - val_accuracy: 0.5000
Epoch 6/100
25/25 [==============================] - 0s 4ms/step - loss: 1.4851 - accuracy: 0.4600 - val_loss: 1.4713 - val_accuracy: 0.3500
Epoch 7/100
25/25 [==============================] - 0s 4ms/step - loss: 1.4304 - accuracy: 0.4550 - val_loss: 1.4184 - val_accuracy: 0.4300
Epoch 8/100
25/25 [=====

In [156]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

1/7 [===>..........................] - ETA: 0s - loss: 0.3132 - accuracy: 0.9062

7/7 [==============================] - 0s 3ms/step - loss: 0.2744 - accuracy: 0.9050
Test Accuracy: 0.9049999713897705


In [157]:
# Save the model and label mapping
model.save('sign_language_model.h5')

In [158]:
# Create a dictionary mapping class labels to class names
label_to_class_name = {label_to_int[label]: label for label in label_to_int}

Testing Data

In [159]:
# Save the label mapping to a JSON file
with open('label_mapping.json', 'w') as json_file:
    json.dump(label_to_class_name, json_file)

In [160]:
# Load the trained model
model = tf.keras.models.load_model('sign_language_model.h5')

In [161]:
# Load the label information from the JSON file
with open('label_mapping.json', 'r') as json_file:
    label_mapping = json.load(json_file)

In [162]:
# Path to your test landmarks YAML file
test_landmarks_path = 'Signs/Hello/left_10.yml'  # Replace with the path to your test landmarks YAML file

In [163]:
# Function to preprocess hand landmarks
def preprocess_hand_landmarks(hand_landmarks):
    landmarks = [point['x'] for point in hand_landmarks['hand_landmarks']]
    return np.array(landmarks)

In [164]:
# Load hand landmarks data from YAML file
with open(test_landmarks_path, 'r') as yaml_file:
    hand_landmarks_data = yaml.safe_load(yaml_file)

In [165]:
# Preprocess hand landmarks
hand_landmarks_processed = preprocess_hand_landmarks(hand_landmarks_data)


In [166]:
# Resize the data to match the input size expected by the model
hand_landmarks_resized = np.expand_dims(hand_landmarks_processed, axis=0)


In [167]:
# Make prediction
prediction = model.predict(hand_landmarks_resized)
predicted_class = np.argmax(prediction)
print(predicted_class)


1/1 [==============================] - 0s 64ms/step
4


In [168]:
# Map the predicted class to its corresponding class name
predicted_class_name = label_mapping.get(str(predicted_class), "Unknown")

# Extract only the last part of the path (class name)
predicted_class_name = os.path.basename(predicted_class_name)

In [169]:
# Display the predicted class name
print(f"Predicted Class Name: {predicted_class_name}")

Predicted Class Name: Hello
